In [1]:
import onnx, torch
from onnx2pytorch import ConvertModel
import numpy as np
import yaml
import torchaudio
import tf2onnx

In [2]:
with open('configs/run_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [3]:
effnet_model = onnx.load("model/discogs-effnet-bsdynamic-1.onnx")
effnet_model = ConvertModel(effnet_model)
effnet_model.eval()

c:\Users\pafitis\envs\digger\Lib\site-packages\onnx2pytorch\convert\layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:209.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))


ConvertModel(
  (Unsqueeze_361): Unsqueeze()
  (Conv_651): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Sigmoid_364): Sigmoid()
  (Mul_365): mul()
  (Conv_654): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
  (Sigmoid_368): Sigmoid()
  (Mul_369): mul()
  (GlobalAveragePool_370): GlobalAveragePool()
  (Conv_371): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
  (Sigmoid_372): Sigmoid()
  (Mul_373): mul()
  (Conv_374): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
  (Sigmoid_375): Sigmoid()
  (Mul_376): mul()
  (Conv_657): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
  (Conv_660): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
  (Sigmoid_381): Sigmoid()
  (Mul_382): mul()
  (Conv_663): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=96)
  (Sigmoid_385): Sigmoid()
  (Mul_386): mul()
  (GlobalAveragePool_387): GlobalAveragePool()
  (Conv_388): Conv2d(96, 4, kernel_size=(1, 1), stride=(1, 1))
  (Sigm

In [4]:
wave, sr = torchaudio.load('temp/Befog - Fractus [fK7FXVqcmQU].wav')
waveform = torchaudio.functional.resample(wave, sr, config.get('MODEL_SR'))
waveform = torch.mean(waveform, dim=0, keepdim=True)

In [5]:
def audio_slicer(audio, sample_rate = 16000, window_length = 64):
    # expects a SR of 1600; window_length 64 seconds
    sliced = audio.split(sample_rate * window_length, dim=1)

    # pad to have consistent shapes
    # (left padding, right padding)
    # (0, SR * WL - Slice length) pads right side with 0's for remainder of window
    padded = [torch.nn.functional.pad(slice, (0, sample_rate * window_length - slice.shape[1])).reshape(1,1,-1) for slice in sliced]
    return padded

def _embed(audio, model, cuda=True):
    if cuda:
        model.to('cuda')
        _, out = model(audio.to('cuda'))
    else:
        _, out = model(audio)
    return out

def get_audio_embeddings(audio, model, cuda=True):
    sliced = audio_slicer(audio)
    embeddings = [_embed(slice, model, cuda) for slice in sliced]
    return embeddings
